In [1]:
import pandas as pd

In [2]:
! git clone https://github.com/andrealodi/ORIE-5213-Spring2023.git

Cloning into 'ORIE-5213-Spring2023'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 553 (delta 2), reused 0 (delta 0), pack-reused 533
Receiving objects: 100% (553/553), 3.15 MiB | 21.23 MiB/s, done.
Resolving deltas: 100% (505/505), done.


In [3]:
initial_inven= pd.read_json("ORIE-5213-Spring2023/Bike sharing/Dataset/Initial_Inven.json")

In [4]:
initial_inven

,0
0,7
1,0
2,1
3,9
4,0
5,9
6,5
7,18
8,3
9,5


In [5]:
# number of bikes initially at stand 20
initial_inven.iloc[20]

0    13
Name: 20, dtype: int64

In [6]:
simu0= pd.read_json("ORIE-5213-Spring2023/Bike sharing/Dataset/simu0_0.json")

In [7]:
simu0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       1458 non-null   int64
 1   1       1458 non-null   int64
 2   2       1458 non-null   int64
 3   3       1458 non-null   int64
dtypes: int64(4)
memory usage: 45.7 KB


In [8]:
simu0.describe()

,0,1,2,3
count,1458.000000,1458.000000,1458.000000,1458.000000
mean,834.290123,14.154321,846.270919,13.951989
std,293.686675,9.045841,295.914786,9.105823
min,4.000000,0.000000,1.000000,0.000000
25%,553.750000,6.000000,565.250000,5.000000
50%,903.500000,15.000000,911.500000,14.000000
75%,1060.750000,22.000000,1077.000000,22.000000
max,1439.000000,29.000000,1429.000000,29.000000


In [9]:
simu0

,0,1,2,3
0,4,1,22,11
1,15,1,37,18
2,19,12,45,8
3,48,11,72,5
4,50,22,73,27
...,...,...,...,...
1453,1414,27,3,9
1454,1417,17,1428,3
1455,1432,0,14,8
1456,1439,9,21,29


In [10]:
# Adjust column 2 based on values in column 0
simu0.loc[simu0[0] > simu0[2], 2] += 1440


In [11]:
simu0

,0,1,2,3
0,4,1,22,11
1,15,1,37,18
2,19,12,45,8
3,48,11,72,5
4,50,22,73,27
...,...,...,...,...
1453,1414,27,1443,9
1454,1417,17,1428,3
1455,1432,0,1454,8
1456,1439,9,1461,29


In [12]:
import pandas as pd

# Define the time ranges
time_range = [(i, i + 15) for i in range(0, 1440, 15)]

# Create the function to assign the time range
def assign_time_range(value):
    for start, end in time_range:
        if start <= value < end:
            return f"{start}-{end}"
    return "Unknown"

# Create the 'departure_time' column
simu0['departure_time'] = simu0[0].apply(assign_time_range)
simu0['arrival_time'] = simu0[2].apply(assign_time_range)

In [13]:
simu0.groupby([1,'departure_time'])[0].count().reset_index()


,1,departure_time,0
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
...,...,...,...
948,29,855-870,1
949,29,915-930,1
950,29,930-945,2
951,29,960-975,3


In [16]:
df_rentals = simu0.groupby([1, 'departure_time'])[0].count().reset_index()
df_rentals = df_rentals.rename(columns={0:"rentals", 1: "station_id"})
df_rentals

,station_id,departure_time,rentals
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
...,...,...,...
948,29,855-870,1
949,29,915-930,1
950,29,930-945,2
951,29,960-975,3


In [17]:
df_returns = simu0.groupby([3, 'arrival_time'])[2].count().reset_index()
df_returns = df_returns.rename(columns={2:"returns", 3: "station_id"})
df_returns

,station_id,arrival_time,returns
0,0,1065-1080,1
1,0,1080-1095,1
2,0,1095-1110,1
3,0,1125-1140,1
4,0,1140-1155,2
...,...,...,...
952,29,945-960,1
953,29,960-975,3
954,29,975-990,1
955,29,990-1005,1


In [54]:
import os
import pandas as pd

folder_path = "ORIE-5213-Spring2023/Bike sharing/Dataset"
json_files = [file for file in os.listdir(folder_path) if (file.endswith(".json") & file.startswith("simu"))]

dataframes = []
for json_file in json_files:
    day = json_file.split("_")[1].split(".")[0]
    file_path = os.path.join(folder_path, json_file)
    df = pd.read_json(file_path)
    df["day"] = int(day)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True).sort_values(by='day')


In [56]:
combined_df.head(20)

,0,1,2,3,day
392122,1052,29,1077,7,0
392016,1024,26,1039,21,0
392015,1024,19,1029,22,0
392014,1024,13,1053,28,0
392013,1024,3,1051,13,0
392012,1024,2,1050,19,0
392011,1024,0,1052,27,0
392010,1023,18,1050,29,0
392009,1023,10,1046,16,0
392008,1023,4,1049,6,0


In [57]:
combined_df['dt'] = ""
combined_df['at'] = ""

for index, row in combined_df.iterrows():
    if row['day'] != 0:
        dt = row[0] + (1440 * row['day'])
        at = row[2] + (1440 * row['day'])
    else:
        dt = row[0]
        at = row[2]
    combined_df.at[index, 'dt'] = dt
    combined_df.at[index, 'at'] = at


In [61]:
# pd.set_option('display.max_rows', None)

In [62]:
combined_df[(combined_df['day']==0) & (combined_df[2] < combined_df[0])]

,0,1,2,3,day,dt,at
392504,1439,16,7,17,0,1439,7
392503,1439,9,21,29,0,1439,21
392502,1432,0,14,8,0,1432,14
392500,1414,27,3,9,0,1414,3
392499,1412,21,1,9,0,1412,1


In [63]:
# Adjust column 2 based on values in column 0
combined_df.loc[combined_df[0] > combined_df[2], 2] += 1440


In [64]:
combined_df[(combined_df['day']==0) & (combined_df[2] < combined_df[0])]

,0,1,2,3,day,dt,at


In [66]:
import pandas as pd

# Define the time ranges
time_range = [(i, i + 15) for i in range(0, 1440, 15)]

# Create the function to assign the time range
def assign_time_range(value):
    for start, end in time_range:
        if start <= value < end:
            return f"{start}-{end}"
    return "Unknown"

# Create the 'departure_time' column
combined_df['departure_time'] = combined_df[0].apply(assign_time_range)
combined_df['arrival_time'] = combined_df[2].apply(assign_time_range)

In [ ]:
combined_df[combined_df['day']==0]

In [70]:
df_rentals = simu0.groupby([1, 'departure_time'])[0].count().reset_index()
df_rentals = df_rentals.rename(columns={0:"rentals", 1: "station_id"})
df_rentals

,station_id,departure_time,rentals
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
5,0,1080-1095,2
6,0,1095-1110,5
7,0,1125-1140,3
8,0,1155-1170,1
9,0,1170-1185,2


In [71]:
df_returns = simu0.groupby([3, 'arrival_time'])[2].count().reset_index()
df_returns = df_returns.rename(columns={2:"returns", 3: "station_id"})
df_returns

,station_id,arrival_time,returns
0,0,1065-1080,1
1,0,1080-1095,1
2,0,1095-1110,1
3,0,1125-1140,1
4,0,1140-1155,2
5,0,1155-1170,1
6,0,1185-1200,1
7,0,1200-1215,1
8,0,1275-1290,1
9,0,1365-1380,1


In [72]:
df_rentals.to_csv("rentals.csv")
df_returns.to_csv("returns.csv")